In [ ]:
dataset_link = 'https://www.kaggle.com/datasets/zainuddin123/parallel-corpus-for-english-urdu-language'

In [2]:
with open('/content/english-corpus.txt', 'r') as file:
    english_rows = file.readlines()
english_row_count = len(english_rows)

with open('/content/urdu-corpus.txt', 'r') as file:
    urdu_rows = file.readlines()
urdu_row_count = len(urdu_rows)


print(f"Urdu: {urdu_row_count}")
print(f"English: {english_row_count}")

Urdu: 24525
English: 24525


### Working

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

with open('/content/english-corpus.txt', 'r') as file:
    english_sentences = file.readlines()

with open('/content/urdu-corpus.txt', 'r') as file:
    urdu_sentences = file.readlines()

def preprocess(sentences):
    sentences = [s.strip().lower() for s in sentences]
    return sentences

english_sentences = preprocess(english_sentences)
urdu_sentences = preprocess(urdu_sentences)

english_tokenizer = Tokenizer()
english_tokenizer.fit_on_texts(english_sentences)
english_sequences = english_tokenizer.texts_to_sequences(english_sentences)

urdu_tokenizer = Tokenizer()
urdu_tokenizer.fit_on_texts(urdu_sentences)
urdu_sequences = urdu_tokenizer.texts_to_sequences(urdu_sentences)

max_english_length = max([len(seq) for seq in english_sequences])
max_urdu_length = max([len(seq) for seq in urdu_sequences])
english_padded = pad_sequences(english_sequences, maxlen=max_english_length, padding='post')
urdu_padded = pad_sequences(urdu_sequences, maxlen=max_urdu_length, padding='post')
english_padded = np.array(english_padded)
urdu_padded = np.array(urdu_padded)

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

latent_dim = 256

encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(len(english_tokenizer.word_index) + 1, latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(len(urdu_tokenizer.word_index) + 1, latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(urdu_tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            1453824   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 256)            1518080   ['input_2[0][0]']             
                                                                                              

In [5]:
urdu_padded_shifted = np.zeros_like(urdu_padded)
urdu_padded_shifted[:, :-1] = urdu_padded[:, 1:]

model.fit([english_padded, urdu_padded], urdu_padded_shifted,
          batch_size=64,
          epochs=20,
          validation_split=0.2
)

Epoch 1/20
307/307 [==============================] - 14s 31ms/step - loss: 1.6255 - val_loss: 1.3408
Epoch 2/20
307/307 [==============================] - 5s 15ms/step - loss: 1.3104 - val_loss: 1.2913
Epoch 3/20
307/307 [==============================] - 4s 14ms/step - loss: 1.2619 - val_loss: 1.2471
Epoch 4/20
307/307 [==============================] - 5s 16ms/step - loss: 1.2223 - val_loss: 1.2282
Epoch 5/20
307/307 [==============================] - 4s 13ms/step - loss: 1.1823 - val_loss: 1.1784
Epoch 6/20
307/307 [==============================] - 4s 13ms/step - loss: 1.1389 - val_loss: 1.1352
Epoch 7/20
307/307 [==============================] - 5s 15ms/step - loss: 1.0986 - val_loss: 1.1228
Epoch 8/20
307/307 [==============================] - 4s 13ms/step - loss: 1.0622 - val_loss: 1.0764
Epoch 9/20
307/307 [==============================] - 4s 13ms/step - loss: 1.0296 - val_loss: 1.0458
Epoch 10/20
307/307 [==============================] - 5s 16ms/step - loss: 0.9975 - val_l

In [ ]:
# Adversarial Training
from tensorflow.keras.layers import GRU, TimeDistributed
from tensorflow.keras.optimizers import Adam

discriminator = tf.keras.Sequential([
    Embedding(len(urdu_tokenizer.word_index) + 1, latent_dim),
    GRU(latent_dim, return_sequences=True),
    TimeDistributed(Dense(1, activation='sigmoid'))
])

discriminator.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy')

def train_discriminator(model, english_data, urdu_data, batch_size=64):
    translated_urdu = model.predict([english_data, urdu_data])
    real_labels = np.ones((batch_size, max_urdu_length, 1))
    fake_labels = np.zeros((batch_size, max_urdu_length, 1))
    discriminator.train_on_batch(urdu_data, real_labels)
    discriminator.train_on_batch(translated_urdu, fake_labels)

epochs = 20
for epoch in range(epochs):
    model.fit([english_padded, urdu_padded], urdu_padded_shifted, batch_size=64, epochs=1)
    train_discriminator(model, english_padded, urdu_padded)
    print(f"Epoch {epoch+1}/{epochs} completed")

In [ ]:
# Eval
from nltk.translate.bleu_score import sentence_bleu

def evaluate_model(model, input_seq, target_seq):
    prediction = model.predict(input_seq)
    predicted_seq = np.argmax(prediction, axis=-1)
    for i in range(len(input_seq)):
        input_sentence = ' '.join([english_tokenizer.index_word[word] for word in input_seq[i] if word != 0])
        target_sentence = ' '.join([urdu_tokenizer.index_word[word] for word in target_seq[i] if word != 0])
        predicted_sentence = ' '.join([urdu_tokenizer.index_word[word] for word in predicted_seq[i] if word != 0])
        print(f"Input: {input_sentence}")
        print(f"Target: {target_sentence}")
        print(f"Predicted: {predicted_sentence}")
        print(f"BLEU score: {sentence_bleu([target_sentence.split()], predicted_sentence.split())}")
        print("")

evaluate_model(model, english_padded[:10], urdu_padded[:10])